Load some packages 

In [1]:
%run preample.ipy

Import specific functions 

In [2]:
from Utilities import *
from BiCM import *

# This script solves the BiCM for some of the datasets we are analyzing

## Numerical Solving BiCM
    - Get degree sequences
    - run numerical simulation to solve system
    - calculate the link probabilities
    

In [3]:
def get_degree_sequences(M_hat):
    
    degree_sequence_item = np.sum(M_hat,1)
    degree_sequence_product = np.sum(M_hat,0)
    degree_sequence = list(np.concatenate([degree_sequence_item, degree_sequence_product]))
    n_rows = len(degree_sequence_item)
    n_cols = len(degree_sequence_product)
    return degree_sequence, n_rows, n_cols

In [4]:
def get_link_probabilities(M, solver = 'root'):
    [degree_sequence, n_rows, n_cols] = get_degree_sequences(M)
    bicm = bicm_leastSquares(degree_sequence, n_rows, n_cols)
    if solver == 'root':
        bicm.solve_root()
    if solver == 'least_squares':
        bicm.solve_least_squares()
    if solver == 'trust_region':
        bicm.solve_trust_regions()
    bicm.generate_p_adjacency()
    return bicm.p_adjacency, abs(np.sum(bicm.final_result))

# Movielens

In [5]:
import os
file_dir = '../data/intermediate/movielens/adjacency_link_removed/'

for file in tqdm(os.listdir(file_dir)):
    print(file)
    adj = np.loadtxt(file_dir + file, delimiter=',').astype(int)
    
    original_path = '../data/intermediate/movielens/adjacency/movielens_adjacency_unweighted.csv'
    original_matrix = np.loadtxt(original_path, delimiter=',').astype(int)
    num_removed_links = np.sum(original_matrix) - np.sum(adj)
    
    p_adj, cost = get_link_probabilities(adj)
    print('solved', cost)
    if cost > 1:
        p_adj, cost = get_link_probabilities(adj, 'least_squares')
        print('solvedls', cost)
    if cost > 1:
        print('unsolved', file)
        continue
    np.savetxt('../data/intermediate/movielens/probabilities_adjacency/' + file, p_adj)
    assert cost < 1, 'numerical convergence problems'
    df = sort_probabilities(p_adj, adj)
    R = fill_miss_links(df, adj, num_removed_links, 0)
    np.savetxt('../data/intermediate/movielens/adjacency_reconstructed/' + file, R)
  


  0%|          | 0/10 [00:00<?, ?it/s]

movielens_adjacency_unweighted_iter_9.csv
solved 1437.7869088281318


 10%|█         | 1/10 [00:22<03:19, 22.11s/it]

`xtol` termination condition is satisfied.
Function evaluations 678, initial cost 4.5057e+06, final cost 5.4438e+03, first-order optimality 5.71e+03.
solvedls 123.90854761411191
unsolved movielens_adjacency_unweighted_iter_9.csv
movielens_adjacency_unweighted_iter_0.csv


 20%|██        | 2/10 [00:25<02:12, 16.53s/it]

...

solved 3.9546145392743915e-13
movielens_adjacency_unweighted_iter_4.csv


 90%|█████████ | 9/10 [01:27<00:07,  7.91s/it]

solved 2.636995320509916e-11
movielens_adjacency_unweighted_iter_5.csv
solved 448.7330841840209


100%|██████████| 10/10 [01:52<00:00, 12.91s/it]

`xtol` termination condition is satisfied.
Function evaluations 817, initial cost 4.6683e+06, final cost 5.7506e+03, first-order optimality 2.79e+03.
solvedls 18.071669689541153
unsolved movielens_adjacency_unweighted_iter_5.csv


# World Trade Web

In [1]:
def remove_links(A, frac = 0.1):
    nonzero = zip(*np.nonzero(A))
    len_nonzeros = len(list(nonzero))
    nonzero = list(zip(*np.nonzero(A)))
    N_links_remove = int(len_nonzeros*frac)
    idx_to_remove = np.random.choice(len_nonzeros, N_links_remove, replace=False)

    A_hat = A.copy()
    for idx in idx_to_remove:
        A_hat[nonzero[idx]] = 0
    removed_links = [nonzero[i] for i in idx_to_remove]
    return A_hat, removed_links
 

In [ ]:
adj = np.genfromtxt('../data/raw/Mcp_2000.dat')
adj = adj.astype(int)
np.savetxt('../data/intermediate/wtw/adjacency/wtw_2000.csv', adj)

file_dir = '../data/intermediate/wtw/adjacency_link_removed/'
for i in np.arange(15):
    adj_removed = remove_links(adj)[0]
    print(np.sum(adj))
    print(np.sum(adj_removed))
    np.savetxt(file_dir + 'wtw_2000_' + str(i) + '.csv', adj_removed, delimiter=',')


In [ ]:

import os
file_dir = '../data/intermediate/wtw/adjacency_link_removed/'

for file in tqdm(os.listdir(file_dir)):
    print(file)
    adj = np.loadtxt(file_dir + file).astype(int)
    
    original_path = '../data/intermediate/wtw/adjacency/wtw_2000.csv'
    original_matrix = np.loadtxt(original_path)
    num_removed_links = np.sum(original_matrix) - np.sum(adj)
    
    p_adj, cost = get_link_probabilities(adj)
    print('solved', cost)
    if cost > 1:
        p_adj, cost = get_link_probabilities(adj, 'least_squares')
        print('solvedls', cost)
    if cost > 1:
        print('unsolved', file)
        continue
    np.savetxt('../data/intermediate/wtw/probabilities_adjacency/p_' + file, p_adj)
    assert cost < 1, 'numerical convergence problems'
    df = sort_probabilities(p_adj, adj)
    R = fill_miss_links(df, adj, num_removed_links, 0)
    np.savetxt('../data/intermediate/wtw/adjacency_reconstructed/r_' + file, R)
    #np.savetxt('../data/intermediate/wtw/probabilities_adjacency/p_' + file, p_adj)
    


# Toy Models

In [ ]:
#adj = np.loadtxt('../data/intermediate/toy_models/adjacency/straight_bip.csv', adj)
import os

file_dir = '../data/intermediate/toy_models/adjacency/'
file_dir_rem = '../data/intermediate/toy_models/adjacency_link_removed/'
for file in tqdm(os.listdir(file_dir)):
    print(file)
    print(file_dir + file)
    adj = np.loadtxt(file_dir + file, delimiter=',').astype(int)
    for i in np.arange(15):
        adj_removed = remove_links(adj)[0]
        print(np.sum(adj))
        print(np.sum(adj_removed))
        np.savetxt(file_dir_rem + file[:-4] + '_' +  str(i) + '.csv', adj_removed)


In [ ]:

import os
file_dir = '../data/intermediate/toy_models/adjacency_link_removed/'

for file in tqdm(os.listdir(file_dir)):
    print(file)
    adj = np.loadtxt(file_dir + file).astype(int)
    
    if file[0] == 'b':
        original_path = '../data/intermediate/toy_models/adjacency/blocks_bip.csv' 
        original_matrix = np.loadtxt(original_path, delimiter=',').astype(int)
    if file[0] == 's':
        original_path = '../data/intermediate/toy_models/adjacency/straight_bip.csv' 
        original_matrix = np.loadtxt(original_path, delimiter=',').astype(int)
    if file[0] == 'h':
        original_path = '../data/intermediate/toy_models/adjacency/hight_dense_bip.csv' 
        original_matrix = np.loadtxt(original_path, delimiter=',').astype(int)
    if file[0] == 'l':
        original_path = '../data/intermediate/toy_models/adjacency/low_dense_bip.csv' 
        original_matrix = np.loadtxt(original_path, delimiter=',').astype(int)
        
        
    num_removed_links = np.sum(original_matrix) - np.sum(adj)
    
    p_adj, cost = get_link_probabilities(adj)
    print('solved', cost)
    if cost > 1:
        p_adj, cost = get_link_probabilities(adj, 'least_squares')
        print('solvedls', cost)
    if cost > 1:
        print('unsolved', file)
        continue
    np.savetxt('../data/intermediate/toy_models/probabilities_adjacency/' + file, p_adj)
    assert cost < 1, 'numerical convergence problems'
    df = sort_probabilities(p_adj, adj)
    R = fill_miss_links(df, adj, num_removed_links, 0)
    np.savetxt('../data/intermediate/toy_models/adjacency_reconstructed/' + file, R)
    np.savetxt('../data/intermediate/toy_models/probabilities_adjacency/' + file, p_adj)
    


# Venezuelan Banks

In [ ]:
import os
file_dir = '../data/intermediate/bank/adjacency_link_removed/'

for file in tqdm(os.listdir(file_dir)):
    adj = np.loadtxt(file_dir + file, delimiter=',').astype(int)
    
    original_path = '../data/intermediate/bank/adjacency/' + file[0:9] + file[14:20] + '.csv'
    original_matrix = np.loadtxt(original_path, delimiter=',').astype(int)
    num_removed_links = np.sum(original_matrix) - np.sum(adj)
    
    p_adj, cost = get_link_probabilities(adj)
    if cost > 1:
        p_adj, cost = get_link_probabilities(adj, 'least_squares')
    if cost > 1:
        continue
    #assert cost < 1, 'numerical convergence problems'
    df = sort_probabilities(p_adj, adj)
    R = fill_miss_links(df, adj, num_removed_links, 0)
    np.savetxt('../data/intermediate/bank/adjacency_reconstructed/' + file, R)
    np.savetxt('../data/intermediate/bank/probabilities_adjacency/' + file, p_adj)
    


# Amazon

In [ ]:
import scipy.sparse
#sparse_matrix = scipy.sparse.load_npz('../data/raw/amazon_adjacency_unweighted/amazon_video_games.npz')
sparse_matrix = scipy.sparse.load_npz('../data/raw/amazon_music_adjacency_unweighted/amazon_music_instruments.npz')
sparse_matrix.shape

In [ ]:
M_hat = sparse_matrix.copy()
degree_sequence_item = np.sum(M_hat,1)
degree_sequence_product = np.transpose(np.sum(M_hat,0))
degree_sequence = list(np.concatenate([degree_sequence_item, degree_sequence_product]))
n_rows = len(degree_sequence_item)
n_cols = len(degree_sequence_product)

In [ ]:
bicm = bicm_leastSquares(degree_sequence, n_rows, n_cols)
bicm.final_result

Second try using the first iteration results as initial values

In [ ]:
x0_2 = np.concatenate([np.unique(bicm.x), np.unique(bicm.y)])
bicm.solve_least_squares(initial_guess=x0_2)